In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import re

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
train = pd.read_csv('records.csv') # мой подготовленный датасет для обучения модели
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
#Смотрим собранные Юрием данные
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  description              37426 non-null  object 
 11  mileage                  37426 non-null  int64  
 12  Привод                   37356 non-null  object 
 13  Руль                     37232 non-null  object 
 14  Состояние             

In [4]:
#Имеем 31 колонку и 37426 строчек
#Посмотрим как данные выглядят и может сразу что то уберем
pd.set_option('display.max_columns', 40)
display(train.head(20))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Аудиосистема,Фары,Климат-контроль,Салон,Диски,autoCatalogUrl,itemPrice,marketPrice,status,closingReason,typeOfTrade,canonicalUrl,avitoId
0,внедорожник,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,Прoдaю лeгендаpный aвтомобиль по ценe китайcко...,180000,полный,Левый,Не битый,3,передние и задние,гидро-,NaN,NaN,Кондиционер,велюр,NaN,/autocatalog/land_rover/discovery/i-19891998_4...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/land_ro...,263245368
1,внедорожник,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,"В нормальном состоянии. Салон кожаный, пяти м...",330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/hyundai/galloper/i-19912003/vnedo...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/hyundai...,633133188
2,седан,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,"Отличный автомобиль, всё есть, надо небольшое ...",124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,8+ колонок,NaN,NaN,NaN,"18""",/autocatalog/mercedes-benz/s-klass/w221-200520...,700000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,624926344
3,хэтчбек,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,"В отличном состоянии, комплетация Elegance c д...",76500,передний,Левый,Не битый,1,передние и задние,электро-,NaN,NaN,климат-контроль многозонный,NaN,"16""",/autocatalog/skoda/octavia/iii-20132017_3291/k...,675000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/skoda_o...,624923120
4,микроавтобус,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,"B наличии, бeз пpобeга Меrсеdеs Sprintеr VIP о...",90,задний,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,кожа,NaN,/autocatalog/mercedes-benz/sprinter_2370/w906-...,10900000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,622789649
5,минивэн,Opel,Серый,Бензин,Zafira,5.0,2012,робот,1.8,140 л.с.,"В хорошем состоянии, находится в сочи",111300,передний,Левый,Не битый,2,передние и задние,электро-,4 колонки,NaN,климат-контроль однозонный,NaN,"16""",/autocatalog/opel/zafira/b-restayling-20082014...,385000,444200.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/opel_za...,633420571
6,седан,Mercedes-Benz,Чёрный,Дизель,E-класс,4.0,2004,автоматическая,2.1,150,....ПРИГНАНА ИЗ ГЕРМАНИИ 2008Г...ЖЕЛЕЗО И КРАС...,380000,задний,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/e-klass/w211s211-20...,340000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,634877947
7,седан,Oldsmobile,Красный,Бензин,Eighty-Eight,4.0,1992,автоматическая,3.8,172 л.с.,Машина не нa ходу. Kоробка в идeалe! двиготeль...,150000,передний,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/oldsmobile/eighty-eight/x-1992199...,80000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/oldsmob...,621276255
8,внедорожник,Mercedes-Benz,Золотой,Бензин,M-класс AMG,5.0,2002,автоматическая,5.4,347,Пpодaм Mеpсeдес Мл 320 АMG в кузовe (w163) тех...,160000,полный,Левый,Не битый,1,передние и задние,гидро-,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/m-klass_amg/w163-re...,500000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,627205114
9,внедорожник,Volkswagen,Серый,Дизель,Touareg,5.0,2008,автоматическая,3.0,240 л.с.,Сpoчнo!!! Maшина в отличном техничеcком cоcтоя...,222000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/volkswagen/touareg/i-restayling-2...,715000,796600.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/volkswa...,629324840


In [5]:
#На первый взгляд видим следующее:
#1. enginePower- нужно будет убрать "л.с."
#2. description- предлагаю полностью убрать это столбец
#3. autoCatalogUrl, status, closingReason, canonicalUrl, avitoId, Аудиосистема, Фары - можно спокойно удалять. Такие значения
# как аудиосистема или фары, практически не вляют на ценник, да и информации по ним мало.
#4. С остальными полями будем разбираться дальше.
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

In [6]:
#Проверям, смотрим. Осталось 23 колонки. Теперь пройдемся по каждой
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  mileage                  37426 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37232 non-null  object 
 13  Состояние                37426 non-null  object 
 14  Владельцы             

In [7]:
#Посмотрим какие уникальные значения в столбце bodyType
train.bodyType.value_counts()

седан           14412
внедорожник      9808
хэтчбек          6198
минивэн          1926
универсал        1693
фургон           1591
купе              834
пикап             354
микроавтобус      347
кабриолет         193
Name: bodyType, dtype: int64

In [8]:
#все хорошо есть, но есть 70 пропусков, предлагаю их убрать. Еще одно НО, посмотрим на них полностью
temp = train[train.bodyType.isnull()]
display(temp.head(30))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
1068,NaN,NaN,Фиолетовый,NaN,NaN,NaN,2012,NaN,NaN,117 л.с.,43750,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,650000,NaN,Продаю личный автомобиль
1234,NaN,NaN,Серебряный,NaN,NaN,NaN,2007,NaN,NaN,248 л.с.,195000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,399000,497400.0,Продаю личный автомобиль
1380,NaN,NaN,Красный,NaN,NaN,NaN,1994,NaN,NaN,90 л.с.,198500,NaN,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,87000,94700.0,Автомобиль приобретён на продажу
1448,NaN,NaN,Белый,NaN,NaN,NaN,2011,NaN,NaN,163 л.с.,221000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,810000,957700.0,Продаю личный автомобиль
1480,NaN,NaN,Чёрный,NaN,NaN,NaN,2012,NaN,NaN,170 л.с.,143000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1000000,974300.0,Продаю личный автомобиль
1484,NaN,NaN,Чёрный,NaN,NaN,NaN,2013,NaN,NaN,249 л.с.,77000,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,1100000,1109800.0,Продаю личный автомобиль
1818,NaN,NaN,Чёрный,NaN,NaN,NaN,2014,NaN,NaN,245 л.с.,77000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1630000,NaN,Продаю личный автомобиль
2141,NaN,NaN,Серый,NaN,NaN,NaN,2001,NaN,NaN,65 л.с.,202000,NaN,Правый,Не битый,2,NaN,NaN,NaN,NaN,NaN,80000,NaN,Продаю личный автомобиль
2193,NaN,NaN,Чёрный,NaN,NaN,NaN,2016,NaN,NaN,135 л.с.,35000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,599000,572000.0,Продаю личный автомобиль
2197,NaN,NaN,Синий,NaN,NaN,NaN,2006,NaN,NaN,125 л.с.,156000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,250000,249200.0,Автомобиль приобретён на продажу


In [9]:
#Видим, что информации по ним 0, удаляем
train.dropna(subset = ['bodyType'],inplace = True)

In [10]:
#число полей сократилось на 70
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37356 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37356 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37356 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37122 non-null  object 
 9   enginePower              37340 non-null  object 
 10  mileage                  37356 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37162 non-null  object 
 13  Состояние                37356 non-null  object 
 14  Владельцы             

In [11]:
#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

In [12]:
#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

In [13]:
#Смотрим
train.head()

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
0,2,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,180000,полный,Левый,Не битый,3,передние и задние,гидро-,Кондиционер,велюр,NaN,170000,NaN,Продаю личный автомобиль
1,2,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,170000,NaN,Продаю личный автомобиль
2,1,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,NaN,NaN,"18""",700000,NaN,Продаю личный автомобиль
3,3,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,76500,передний,Левый,Не битый,1,передние и задние,электро-,климат-контроль многозонный,NaN,"16""",675000,NaN,Продаю личный автомобиль
4,9,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,90,задний,Левый,Не битый,1,NaN,NaN,NaN,кожа,NaN,10900000,NaN,Продаю личный автомобиль


# Смотрим "brand"

In [14]:
#Получаем, что у нас 123 различных брендов. Пойдем по пути, что в ТЕСТОВОМ у нас одни BMW
train.brand.value_counts()

Mercedes-Benz      3002
BMW                2519
Volkswagen         2375
Hyundai            2335
ВАЗ (LADA)         2302
                   ... 
Dacia                 1
AMC                   1
McLaren               1
Renault Samsung       1
Dadi                  1
Name: brand, Length: 123, dtype: int64

In [15]:
#Оставим одни значения BMW
train_BMW = train[train['brand'].isin(['BMW'])]

In [16]:
#Проверим 
train_BMW

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
16,1,BMW,Белый,Дизель,5 серия,4.0,1996,механическая,2.9,184 л.с.,240000,задний,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,350000,NaN,Продаю личный автомобиль
20,1,BMW,Серебряный,Бензин,5 серия,4.0,2003,автоматическая,2.5,192 л.с.,380000,задний,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,475000,NaN,Продаю личный автомобиль
38,1,BMW,Чёрный,Дизель,7 серия,4.0,2016,автоматическая,3.0,265 л.с.,109000,полный,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,2750000,2928700.0,Автомобиль приобретён на продажу
46,2,BMW,Чёрный,Бензин,X5,5.0,2004,автоматическая,4.4,320 л.с.,251000,полный,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,615000,NaN,Продаю личный автомобиль
56,2,BMW,Чёрный,Бензин,X6,5.0,2008,автоматическая,3.0,306 л.с.,123700,полный,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,1150000,1106800.0,Продаю личный автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37364,2,BMW,Чёрный,Бензин,X5,5.0,2017,автоматическая,3.0,306 л.с.,39200,полный,Левый,Не битый,2,передние и задние,электрогидро-,NaN,NaN,"19""",2900000,NaN,Продаю личный автомобиль
37368,1,BMW,Серебряный,Бензин,5 серия,4.0,2001,автоматическая,2.2,170 л.с.,290000,задний,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,255000,NaN,Продаю личный автомобиль
37371,2,BMW,Белый,Бензин,X6 M,5.0,2010,автоматическая,4.4,555 л.с.,76000,полный,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,1700000,NaN,Продаю личный автомобиль
37396,7,BMW,Чёрный,Бензин,6 серия,2.0,2016,автоматическая,3.0,320 л.с.,57000,полный,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,2390000,2397200.0,Продаю личный автомобиль


# Столбец "color"

In [17]:
#Количество 37356 пропусков нет
print(train.color.value_counts())
(train.color.value_counts()).sum()

Чёрный        9849
Белый         7659
Серый         4711
Серебряный    3933
Синий         3604
Красный       2083
Зелёный       1374
Коричневый    1287
Бежевый        868
Голубой        561
Жёлтый         436
Золотой        307
Фиолетовый     300
Оранжевый      219
Пурпурный      150
Розовый         15
Name: color, dtype: int64


37356

In [21]:
#Создадим словарь
dict_color={'Чёрный': 1,
'Белый': 2,
'Серый': 3,
'Серебряный': 4,
'Синий': 5,
'Красный': 6,
'Зелёный': 7,
'Коричневый': 8,
'Бежевый': 9,
'Голубой': 10,
'Жёлтый': 11,
'Золотой': 12,
'Фиолетовый': 13,
'Оранжевый': 14,
'Пурпурный': 15,
'Розовый': 16}
train['color'] = train['color'].map(dict_color)
train_BMW['color'] = train_BMW['color'].map(dict_color)

D:\ANACONDA\1223\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
#Проверили сошлось
print(train.color.value_counts())
(train.color.value_counts()).sum()

1     9849
2     7659
3     4711
4     3933
5     3604
6     2083
7     1374
8     1287
9      868
10     561
11     436
12     307
13     300
14     219
15     150
16      15
Name: color, dtype: int64


37356

In [29]:
#Проверили сошлось
print(train_BMW.color.value_counts())
(train_BMW.color.value_counts()).sum()

1     1051
2      402
5      324
3      312
4      130
6      101
8       86
7       33
9       31
10      21
14      11
12       7
13       5
15       4
11       1
Name: color, dtype: int64


2519

# Столбец "fuelType"

In [32]:
print(train.fuelType.value_counts())
print(train_BMW.fuelType.value_counts())

Бензин     31393
Дизель      5684
Гибрид       185
Электро       87
Газ            7
Name: fuelType, dtype: int64
Бензин     1812
Дизель      692
Гибрид       13
Электро       2
Name: fuelType, dtype: int64


In [36]:
#Создаем словарь
dict_fuelType={'Бензин': 1,
'Дизель': 2,
'Гибрид': 3,
'Электро': 4,
'Газ': 5}

In [37]:
train['fuelType'] = train['fuelType'].map(dict_fuelType)
train_BMW['fuelType'] = train_BMW['fuelType'].map(dict_fuelType)

D:\ANACONDA\1223\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
#Проверили, все сошлось
print(train.fuelType.value_counts())
print(train_BMW.fuelType.value_counts())

1    31393
2     5684
3      185
4       87
5        7
Name: fuelType, dtype: int64
1    1812
2     692
3      13
4       2
Name: fuelType, dtype: int64


# Столбцы 'numberOfDoors', "productionDate","engineDisplacement", "mileage"

In [40]:
print(train.numberOfDoors.value_counts())


5.0    18473
4.0    15777
2.0     1933
3.0     1172
Name: numberOfDoors, dtype: int64
